In [1]:
import caffe
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import lmdb
import os
import caffe.draw
from caffe.proto import caffe_pb2
from google.protobuf import text_format

In [2]:
caffe.set_mode_cpu()

In [4]:
network_inputs_dir = u'E:\\Diploma\\Network_inputs\\'
network_outputs_dir = u'E:\\Diploma\\Network_outputs\\'

simhei = 'Simhei'
simsun = 'Simsun'

In [6]:
import random

random_permutation = range(0, len(os.listdir(network_inputs_dir)))
random.shuffle(random_permutation)

print len(random_permutation)
train_size = 700
test_size = 300 #len(random_permutation) - train_size
train_indices = random_permutation[:train_size]
test_indices = random_permutation[train_size: train_size + test_size]
print len(train_indices), len(test_indices)

25795
700 300


In [8]:
#height, width
def pack_lmdb(source_dir, lmdb_name, indices):
    
    dataset = lmdb.open(lmdb_name)
    with dataset.begin(write=True) as ds:
        filenames = os.listdir(source_dir)
        for index in indices:
            image = np.array(Image.open(source_dir + filenames[index])).astype(np.uint8)
            image = image[np.newaxis]
            image_raw = caffe.io.array_to_datum(image)
            ds.put('{:0>10d}'.format(index), image_raw.SerializeToString())
    dataset.close()

In [9]:
pack_lmdb(network_outputs_dir + simhei + '\\', 'label_simhei_train', train_indices)
pack_lmdb(network_inputs_dir, 'data_train', train_indices)
pack_lmdb(network_outputs_dir + simhei + '\\', 'label_simhei_test', test_indices)
pack_lmdb(network_inputs_dir, 'data_test', test_indices)

In [11]:
net = caffe_pb2.NetParameter()
text_format.Merge(open('net_glyph.prototxt').read(), net)
print('Drawing net to %s' % 'my_net.png')

phase = caffe.TRAIN
caffe.draw.draw_net_to_file(net, 'my_net.png', 'LR', phase)

Drawing net to my_net.png


In [3]:
net = caffe.Net('net_glyph.prototxt', caffe.TRAIN)

In [ ]:
solver = caffe.SGDSolver('solver_glyph.prototxt')